In [1]:
# Auto-reload
%load_ext autoreload
%autoreload 2

In [2]:
import re
from collections import defaultdict
from dataclasses import dataclass, field
from pathlib import Path
from typing import Callable, Generator, Optional

import pandas as pd
import numpy as np
import numpy.typing as npt
import matplotlib.pyplot as plt
from aocd import submit

from tools import (
    get_daily_input,
    split_data,
    split_data_2_df,
    Grid,
    GridWithNetwork,
    get_palette,
    pairwise,
    show_palette,
    get_color_from_pct,
    Point,
)

YEAR = 2024

# Day 1


In [3]:
%get_daily_input 1 2024

In [6]:
def solve_day1_a(data_input: str) -> int:
    df = split_data_2_df(data_input, as_int=True)
    return (
        (df[0].sort_values(ignore_index=True) - df[1].sort_values(ignore_index=True))
        .abs()
        .sum()
    )


EXAMPLE = """
3   4
4   3
2   5
1   3
3   9
3   3
"""

solve_day1_a(EXAMPLE)

11

In [62]:
solution = solve_day1_a(data)
solution

1189304

In [ ]:
# submit(solution, part="a", day=1, year=YEAR)

coerced int64 value 1189304 for 2024/01 to '1189304'


That's the right answer!  You are one gold star closer to finding the Chief Historian. [Continue to Part Two]


In [64]:
# list_1, list_2 = split_data_tuple_lists(data, tuple_separator=r" +", safe_eval=True)
# list_1

## D1 Part B

In [65]:
df = pd.DataFrame([b.split("   ") for b in split_data(EXAMPLE)]).astype(int)
df

,0,1
0,3,4
1,4,3
2,2,5
3,1,3
4,3,9
5,3,3


In [69]:
df.apply(lambda x: x[0] * (df[1] == x[0]).sum(), axis=1).sum()

31

In [7]:
def similarity_score(data_input: str) -> int:
    """Sum nb in left list by nb of occurrences in right list"""
    df = split_data_2_df(data_input, as_int=True)
    return df.apply(lambda x: x[0] * (df[1] == x[0]).sum(), axis=1).sum()

solution = similarity_score(data)
solution


24349736

In [ ]:
# submit(solution, part="b", day=1, year=YEAR)

coerced int64 value 24349736 for 2024/01 to '24349736'


That's the right answer!  You are one gold star closer to finding the Chief Historian.You have completed Day 1! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


# Day 2


In [38]:
%get_daily_input 2 2024
data[:100]

'42 44 47 49 51 52 54 52\n24 27 30 31 32 35 36 36\n80 82 85 86 87 90 94\n4 5 7 10 13 14 20\n38 41 40 42 4'

In [12]:
EXAMPLE = """
7 6 4 2 1
1 2 7 8 9
9 7 6 2 1
1 3 2 4 5
8 6 4 4 1
1 3 6 7 9
"""

In [13]:
df = split_data_2_df(EXAMPLE, as_int=True)
df

,0,1,2,3,4
0,7,6,4,2,1
1,1,2,7,8,9
2,9,7,6,2,1
3,1,3,2,4,5
4,8,6,4,4,1
5,1,3,6,7,9


In [76]:
def is_safe(s: pd.Series) -> bool:
    return (
        (s.is_monotonic_increasing | s.is_monotonic_decreasing)
        & (s != s.shift(1)).all()
        & ((s - s.shift(1)).dropna().abs() <= 3).all()
    )


def solve_day2_a(data_input: str) -> int:
    series = pd.Series(
        [pd.Series(b.split(" ")).astype(int) for b in split_data(data_input)]
    )
    return series.apply(is_safe).sum()


solve_day2_a(EXAMPLE)

2

In [50]:
solution = solve_day2_a(data)
solution

282

In [ ]:
# submit(solution, part="a", day=2, year=YEAR)

coerced int64 value 282 for 2024/02 to '282'


That's the right answer!  You are one gold star closer to finding the Chief Historian. [Continue to Part Two]


## D2 Part B

In [ ]:
def is_safe_with_dampener(s: pd.Series) -> bool:
    """Is safe with a maximum of 1 value removed"""
    series_is_safe = is_safe(s)
    if not series_is_safe:
        # We check if series can be safe by removing one value
        return any(is_safe(s.copy().drop(i)) for i in range(len(s)))
    return True


def solve_day2_b(data_input: str) -> int:
    series = pd.Series(
        [pd.Series(b.split(" ")).astype(int) for b in split_data(data_input)]
    )
    return series.apply(is_safe_with_dampener).sum()


solve_day2_b(EXAMPLE)

4

In [79]:
solution = solve_day2_b(data)
solution

349

In [ ]:
# submit(solution, part="b", day=2, year=YEAR)

coerced int64 value 349 for 2024/02 to '349'


That's the right answer!  You are one gold star closer to finding the Chief Historian.You have completed Day 2! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


# Day 3

In [84]:
%get_daily_input 3 2024
data[:100]

"-~who()?!-{ where()mul(764,406)?^why()%[how(420,460)mul(69,497)where();'&>-!when()<^mul(629,650)mul("

In [85]:
EXAMPLE = "xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))"

In [ ]:
def solve_day3_a(data_input: str) -> int:
    return sum(
        int(d1) * int(d2) for d1, d2 in re.findall(r"mul\((\d+),(\d+)\)", data_input)
    )

solve_day3_a(EXAMPLE)

161

In [92]:
solution = solve_day3_a(data)
solution

178886550

In [89]:
submit(solution, part="a", day=3, year=YEAR)

That's the right answer!  You are one gold star closer to finding the Chief Historian. [Continue to Part Two]


## D3 Part B

In [96]:
EXAMPLE = "xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))"

In [144]:
def solve_day3_b(data_input: str) -> int:
    return sum(
        solve_day3_a(d)
        for d in re.split(r"don't\(\).*?do\(\)", data_input, flags=re.S)
    )

solve_day3_b(EXAMPLE)

0

In [111]:
solution = solve_day3_b(data)
solution

87163705

In [ ]:
# submit(solution, part="b", day=3, year=YEAR)

That's the right answer!  You are one gold star closer to finding the Chief Historian.You have completed Day 3! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


# Day 4

In [113]:
%get_daily_input 4 2024

In [145]:
EXAMPLE="""
....XXMAS.
.SAMXMS...
...S..A...
..A.A.MS.X
XMASAMX.MM
X.....XA.A
S.S.S.S.SS
.A.A.A.A.A
..M.M.M.MM
.X.X.XMASX
"""

In [146]:
# We need to find all XMAS in the grid, horizontally, vertically and diagonally in all directions
# We will use a sliding window of size 4
# We will use a 2D convolution to find the XMAS pattern

def solve_day4_a(data_input: str, pattern: str = "XMAS") -> list[list[Point]]:
    grid = Grid(data_input)
    return grid.find_pattern(pattern)

sol_ = solve_day4_a(EXAMPLE)
print(len(sol_))
sol_

18


[[Point(x=0, y=4), Point(x=1, y=5), Point(x=2, y=6), Point(x=3, y=7)],
 [Point(x=0, y=5), Point(x=0, y=6), Point(x=0, y=7), Point(x=0, y=8)],
 [Point(x=1, y=4), Point(x=1, y=3), Point(x=1, y=2), Point(x=1, y=1)],
 [Point(x=3, y=9), Point(x=4, y=8), Point(x=5, y=7), Point(x=6, y=6)],
 [Point(x=3, y=9), Point(x=4, y=9), Point(x=5, y=9), Point(x=6, y=9)],
 [Point(x=4, y=0), Point(x=4, y=1), Point(x=4, y=2), Point(x=4, y=3)],
 [Point(x=4, y=6), Point(x=3, y=6), Point(x=2, y=6), Point(x=1, y=6)],
 [Point(x=4, y=6), Point(x=4, y=5), Point(x=4, y=4), Point(x=4, y=3)],
 [Point(x=5, y=0), Point(x=4, y=1), Point(x=3, y=2), Point(x=2, y=3)],
 [Point(x=5, y=6), Point(x=4, y=5), Point(x=3, y=4), Point(x=2, y=3)],
 [Point(x=9, y=1), Point(x=8, y=2), Point(x=7, y=3), Point(x=6, y=4)],
 [Point(x=9, y=3), Point(x=8, y=4), Point(x=7, y=5), Point(x=6, y=6)],
 [Point(x=9, y=3), Point(x=8, y=2), Point(x=7, y=1), Point(x=6, y=0)],
 [Point(x=9, y=5), Point(x=8, y=6), Point(x=7, y=7), Point(x=6, y=8)],
 [Poin

In [124]:
first_pattern = sol_[0]
grid = Grid(EXAMPLE)
[grid.data[i][j] for i, j in first_pattern]

['X', 'M', 'A', 'S']

In [126]:
xmas_found = solve_day4_a(data)
solution = len(xmas_found)
solution

2685

In [127]:
submit(solution, part="a", day=4, year=YEAR)

That's the right answer!  You are one gold star closer to finding the Chief Historian. [Continue to Part Two]


# D4 Part B

In [147]:
# Finding all MAS crossing each other diagonally around a A
EXAMPLE = """
.M.S......
..A..MSMS.
.M.S.MAA..
..A.ASMSM.
.M.S.M....
..........
S.S.S.S.S.
.A.A.A.A..
M.M.M.M.M.
..........
"""
grid = Grid(EXAMPLE)
grid

[['.' 'M' '.' 'S' '.' '.' '.' '.' '.' '.']
 ['.' '.' 'A' '.' '.' 'M' 'S' 'M' 'S' '.']
 ['.' 'M' '.' 'S' '.' 'M' 'A' 'A' '.' '.']
 ['.' '.' 'A' '.' 'A' 'S' 'M' 'S' 'M' '.']
 ['.' 'M' '.' 'S' '.' 'M' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['S' '.' 'S' '.' 'S' '.' 'S' '.' 'S' '.']
 ['.' 'A' '.' 'A' '.' 'A' '.' 'A' '.' '.']
 ['M' '.' 'M' '.' 'M' '.' 'M' '.' 'M' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.']]

In [148]:
def solve_day4_b(data_input: str) -> int:
    grid = Grid(data_input)
    matches = grid.find_pattern("MAS", horizontal=False, vertical=False, diagonal=True)
    count_mas = defaultdict(list)
    for m in matches:
        count_mas[m[1]].append(m)
    return sum(1 for c in count_mas.values() if len(c) > 1)

solve_day4_b(EXAMPLE)

9

In [149]:
solution = solve_day4_b(data)
solution

2048

In [150]:
submit(solution, part="b", day=4, year=YEAR)

That's the right answer!  You are one gold star closer to finding the Chief Historian.You have completed Day 4! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


In [151]:
from itertools import product
from typing import List, Tuple

class Point:
    def __init__(self, x: int, y: int):
        self.x = x
        self.y = y

class WordSearch:
    def __init__(self, data: List[str]):
        self.data = data
        self.width = len(data[0])
        self.height = len(data)

    def find_pattern(self, pattern: str, diagonal: bool = True, horizontal: bool = True, vertical: bool = True) -> List[List[Point]]:
        found = []
        directions: List[Tuple[int, int]] = []
        if diagonal:
            directions.extend([(1, 1), (1, -1), (-1, 1), (-1, -1)])
        if horizontal:
            directions.extend([(1, 0), (-1, 0)])
        if vertical:
            directions.extend([(0, 1), (0, -1)])

        for i, j in product(range(self.width), range(self.height)):
            for dx, dy in directions:
                found.append(self.find_pattern_direction(pattern, i, j, dx, dy))

        return [f for f in found if f]

    def find_pattern_direction(self, pattern: str, x: int, y: int, dx: int, dy: int) -> List[Point]:
        pattern_len = len(pattern)
        points = []
        for k in range(pattern_len):
            nx, ny = x + k * dx, y + k * dy
            if 0 <= nx < self.width and 0 <= ny < self.height and self.data[ny][nx] == pattern[k]:
                points.append(Point(nx, ny))
            else:
                return []
        return points

# Define the grid
grid = [
    "MMMSXXMASM",
    "MSAMXMSMSA",
    "AMXSXMAAMM",
    "MSAMASMSMX",
    "XMASAMXAMM",
    "XXAMMXXAMA",
    "SMSMSASXSS",
    "SAXAMASAAA",
    "MAMMMXMMMM",
    "MXMXAXMASX"
]

# Create a WordSearch object
word_search = WordSearch(grid)

# Find all occurrences of "XMAS"
occurrences = word_search.find_pattern("XMAS")

# Print the number of occurrences
print(f"XMAS appears {len(occurrences)} times.")

XMAS appears 18 times.
